<a href="https://colab.research.google.com/github/AlcidesOxa/datos/blob/master/Importing_preparing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paquetes para ciencia de datos

In [ ]:
import pandas as pd
import numpy as np

# Importando datos

In [ ]:
Empleados=pd.read_csv("https://raw.githubusercontent.com/AlcidesOxa/datos/master/ejemplo.csv",sep=';',encoding="ISO-8859-1")
Empleados.head()
Empleados.tail()

In [ ]:
Empleados.columns.values
cambio={'Años estudio':'Anios_estudio', 'Estado Civil':'Estado_civil', 'Satisfacción Trabajo':'Satisfaccion_trabajo', 'Fecha contrato':'Fecha_contrato'}
Empleados.rename(columns=cambio,inplace=True)

# Identificación y manejo de datos perdidos

In [ ]:
print(Empleados.shape,Empleados.dropna().shape,Empleados.dropna(axis=1).shape)

In [ ]:
(Empleados.isnull().sum()/Empleados.isnull().count()).sort_values(ascending = False)
(Empleados.isnull().sum(axis=1)/Empleados.isnull().count(axis=1)).sort_values(ascending = False)

In [ ]:
casosborrar=[3,5,8]
Empleados=Empleados.drop(casosborrar)
varborrar=['Faltas','Permisos']
Empleados=Empleados.drop(varborrar,axis=1)

In [ ]:
Empleados[Empleados.isnull().sum(axis=1)!=0]

In [ ]:
varcategoricas=['Sexo','Estado_civil','Satisfaccion_trabajo']
Empleados[varcategoricas]=Empleados[varcategoricas].fillna('Desconocido')

# Formateando datos

In [ ]:
cod=Empleados.Nombres.str[0]+Empleados.Apellidos.str[0]+Empleados.Fecha_contrato.str.slice(-4)
Empleados.set_index(cod,inplace=True)
Empleados.index.values

In [ ]:
Empleados.dtypes
Empleados.info()

In [ ]:
Empleados[varcategoricas]=Empleados[varcategoricas].astype('category')
Empleados['Fecha_contrato']=pd.to_datetime(Empleados.Fecha_contrato)

In [ ]:
Empleados.Sexo.cat.categories.values
orden_ec=Empleados.Estado_civil.cat.categories.values[[3,0,2,4,1]].tolist()
orden_st=Empleados.Satisfaccion_trabajo.cat.categories.values[[2,3,0,1]].tolist()
Empleados['Estado_civil']=Empleados.Estado_civil.cat.reorder_categories(orden_ec)
Empleados['Satisfaccion_trabajo']=Empleados.Satisfaccion_trabajo.cat.reorder_categories(orden_st)
Empleados['Satisfaccion_trabajo']=Empleados.Satisfaccion_trabajo.cat.rename_categories(
    ['Insatisfecho(a)','Neutral','Satisfecho(a)','Desconocido'])

# Localización de datos

In [ ]:
variables=['Apellidos','Nombres','Edad','Estado_civil','Fecha_contrato']
Empleados[variables]
Empleados.loc[:,variables]
casos=['VC2009','MF2009','PG2013','AM2015']
Empleados.loc[casos,:]
Empleados.loc[casos,variables]
Empleados.loc['VC2009':'JG2011','Sexo':'Gastos']

In [ ]:
Empleados.iloc[4:14,4:7]
casos=[3,6,11,22]
variables=[1,4,8]
Empleados.iloc[casos,variables]
Empleados.iloc[:,variables]
Empleados.iloc[casos,:]

# Selección de datos

In [ ]:
Empleados[Empleados.Sexo=='Mujer']
Empleados[(Empleados.Sexo=='Mujer') & (Empleados.Estado_civil=='Soltero(a)')]
condicion1=(Empleados.Sexo=='Mujer') & (Empleados.Estado_civil=='Soltero(a)') | (Empleados.Estado_civil=='Divorciado(a)')
Empleados[condicion1]
condicion2=(Empleados.Sexo=='Mujer') & ((Empleados.Estado_civil=='Soltero(a)') | (Empleados.Estado_civil=='Divorciado(a)'))
Empleados[condicion2]
condicion3=(Empleados.Edad<40) & (Empleados.Fecha_contrato>'2015-01-01')
Empleados[condicion3]
condicion4=(Empleados.Edad<40) & ~(Empleados.Fecha_contrato>'2015-01-01')
Empleados[condicion4]
Empleados[condicion1 | condicion4]

# Transformación y eliminación de variables

In [ ]:
Empleados['Experiencia_lab']=Empleados.Edad-Empleados.Anios_estudio-5
Empleados['Balance']=Empleados.Ingresos-Empleados.Gastos
Empleados['log_exp']=np.log(Empleados.Edad)/np.exp(Empleados.Anios_estudio)
Empleados['log_exp2']=np.log(Empleados.Edad)-np.exp(Empleados.Anios_estudio)
aborrar=['log_exp','log_exp2']
Empleados=Empleados.drop(aborrar,axis=1)

# Normalizaciones

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
SS=StandardScaler()
MM=MinMaxScaler()
varcuantis=['Edad','Anios_estudio','Ingresos','Gastos','Experiencia_lab','Balance']
SS.fit_transform(Empleados[varcuantis])
MM.fit_transform(Empleados[varcuantis])
(Empleados[varcuantis]-Empleados[varcuantis].mean())/Empleados[varcuantis].std()
(Empleados[varcuantis]-Empleados[varcuantis].min())/(Empleados[varcuantis].max()-Empleados[varcuantis].min())

# Binning

In [ ]:
ing_min=int(Empleados.Ingresos.min()-1)
ing_max=int(Empleados.Ingresos.max()+2)
ing_ampl=int((ing_max-ing_min)/10)
intervalos = ["{0} - {1}".format(i+1,i+ing_ampl) for i in range(ing_min,ing_max,ing_ampl)]
Empleados['Ingreso_agrup']=pd.cut(Empleados.Ingresos,range(ing_min,ing_max+ing_ampl,ing_ampl),labels=intervalos)
Empleados[['Ingresos','Ingreso_agrup']]

# Variables indicadoras

In [ ]:
pd.get_dummies(Empleados.Estado_civil)
pd.get_dummies(Empleados.Estado_civil,dummy_na=True)
pd.get_dummies(Empleados[varcategoricas],prefix=['SX_','EC_','SAT_'])